In [16]:
from HelperInfra import get_target_date , last_friday_of_previous_month , last_thursday ,get_data
import pandas as pd
from datetime import datetime

In [2]:
import pandas as pd
database=pd.read_csv('./FinalDataBase.csv')
database
database["Start Date"] = pd.to_datetime(database["Start Date"], format="%d-%m-%Y")
database["End Date"] = pd.to_datetime(database["End Date"], format="%d-%m-%Y")

In [3]:
database[database['Stock']=='MCDOWELL-N']

,Unnamed: 0,Stock,Start Date,End Date,Accord Code,Sr.No.,Company Name,CD_BSE Code,CD_Bse Scrip Name,CD_NSE Symbol,CD_ISIN No


In [ ]:
stock_price=pd.read_csv('./All Close Prices.csv')
stock_price['Date'] = pd.to_datetime(stock_price['Date']).dt.date
stock_price=stock_price[stock_price['Exchange']=='NSE']
stock_price = stock_price.sort_values(by='Date') 
stock_price

,Name,Date,Close,Exchange,Year
7139800,Hybrid Financial Services Ltd.,2001-04-01,2.350,NSE,2001
4647007,EPL Ltd.,2001-04-01,21.285,NSE,2001
197096,AMJ Land Holdings Ltd.,2001-04-01,3.110,NSE,2001
12700823,PCBL Ltd.,2001-04-01,1.600,NSE,2001
16974302,Sri Havisha Hospitality and Infrastructure Ltd.,2001-04-01,0.500,NSE,2001
...,...,...,...,...,...
15923862,Semac Consultants Ltd.,2023-12-21,2376.500,NSE,2023
10259832,Mahanagar Telephone Nigam Ltd.,2023-12-21,32.400,NSE,2023
7252246,IFB Agro Industries Ltd.,2023-12-21,470.550,NSE,2023
14289430,Rane Holdings Ltd.,2023-12-21,1193.300,NSE,2023


In [5]:
stock_price=stock_price[(stock_price['Date']>=datetime(2012,1,1).date()) & (stock_price['Date']<=datetime(2023,1,1).date())]

In [6]:
stockuniverse=pd.read_csv('./MomentumUniverseForEachMonth.csv')

In [7]:
stockuniverse['End'] = pd.to_datetime(stockuniverse['End']).dt.date
stockuniverse['Start'] = pd.to_datetime(stockuniverse['Start']).dt.date

In [8]:
stockuniverse

,Unnamed: 0,Symbol,Start,End,Company Name,Accord Code
0,0,UNITDSPR,2012-12-28,2013-01-31,United Spirits Ltd.,132432
1,1,KTKBANK,2012-12-28,2013-01-31,The Karnataka Bank Ltd.,132652
2,2,DENABANK,2012-12-28,2013-01-31,Dena Bank - (Amalgamated),132121
3,3,AUROPHARMA,2012-12-28,2013-01-31,Aurobindo Pharma Ltd.,124804
4,4,MRF,2012-12-28,2013-01-31,MRF Ltd.,100290
...,...,...,...,...,...,...
3595,3595,ESCORTS,2022-11-25,2022-12-29,Escorts Kubota Ltd.,100495
3596,3596,CUMMINSIND,2022-11-25,2022-12-29,Cummins India Ltd.,100480
3597,3597,UNITDSPR,2022-11-25,2022-12-29,United Spirits Ltd.,132432
3598,3598,TATAMOTORS,2022-11-25,2022-12-29,Tata Motors Ltd.,100570


In [ ]:
import pandas as pd
from datetime import datetime

# Pre-compute `Company Name` mapping
symbol_to_company = database.set_index('Stock')['Company Name'].to_dict()

# Pre-group stock_price by company name
stock_price_grouped = {name: group for name, group in stock_price.groupby('Name')}

# Define function to calculate percentage change
def find_start_end_close(row): 
    # Get date ranges
    prev_date = row['Start']
    current_date = row['End']
    
    # Get company name
    company_name = symbol_to_company.get(row['Symbol'])
    if company_name is None:
        return None, None, None, None, None  # If no mapping found
    
    # Get stock price data for this company
    stock_price_changed = stock_price_grouped.get(company_name)
    if stock_price_changed is None:
        return None, None, None, None, None  # If no stock data found
    
    # Filter relevant rows
    close_price = stock_price_changed[
        (prev_date <= stock_price_changed['Date']) & 
        (stock_price_changed['Date'] <= current_date)
    ]
    
    if not close_price.empty:
        prev_price = close_price['Close'].iloc[0]
        curr_price = close_price['Close'].iloc[-1]
        percentage = (curr_price - prev_price) / prev_price
        return close_price['Date'].iloc[0], curr_price, close_price['Date'].iloc[-1], prev_price, percentage
    else:
        return None, None, None, None, None

# Apply the optimized function to the DataFrame
stockuniverse[['Prev Date', 'Curr Price', 'Curr Date', 'Prev Price', 'Returns']] = stockuniverse.apply(
    find_start_end_close,
    axis=1,
    result_type='expand'
)


stockuniverse = stockuniverse.dropna()
stockuniverse.dropna(inplace=True)
from datetime import timedelta
stockuniverse['Diff'] = (stockuniverse['Curr Date'] - stockuniverse['Prev Date'])


In [11]:
stockuniverse.sort_values(by='End',ascending=True)

,Unnamed: 0,Symbol,Start,End,Company Name,Accord Code,Prev Date,Curr Price,Curr Date,Prev Price,Returns,Diff
0,0,UNITDSPR,2012-12-28,2013-01-31,United Spirits Ltd.,132432,2012-12-28,360.88,2013-01-31,379.71,-0.049590,"34 days, 0:00:00"
29,29,MCLEODRUSS,2012-12-28,2013-01-31,Mcleod Russel India Ltd.,132654,2012-12-28,362.00,2013-01-31,352.60,0.026659,"34 days, 0:00:00"
28,28,AXISBANK,2012-12-28,2013-01-31,Axis Bank Ltd.,132215,2012-12-28,301.03,2013-01-31,271.91,0.107094,"34 days, 0:00:00"
27,27,JUBLFOOD,2012-12-28,2013-01-31,Jubilant FoodWorks Ltd.,215367,2012-12-28,120.06,2013-01-31,129.72,-0.074468,"34 days, 0:00:00"
26,26,HDIL,2012-12-28,2013-01-31,Housing Development And Infrastructure Ltd.,132873,2012-12-28,76.70,2013-01-31,109.95,-0.302410,"34 days, 0:00:00"
...,...,...,...,...,...,...,...,...,...,...,...,...
3571,3571,TATAPOWER,2022-11-25,2022-12-29,Tata Power Company Ltd.,100400,2022-11-25,205.85,2022-12-29,224.10,-0.081437,"34 days, 0:00:00"
3570,3570,HAL,2022-11-25,2022-12-29,Hindustan Aeronautics Ltd.,219494,2022-11-25,2506.55,2022-12-29,2731.55,-0.082371,"34 days, 0:00:00"
3598,3598,TATAMOTORS,2022-11-25,2022-12-29,Tata Motors Ltd.,100570,2022-11-25,385.90,2022-12-29,433.20,-0.109187,"34 days, 0:00:00"
3583,3583,COALINDIA,2022-11-25,2022-12-29,Coal India Ltd.,219300,2022-11-25,221.65,2022-12-29,231.40,-0.042135,"34 days, 0:00:00"


In [12]:
stockuniverse.to_csv('StockUniverseWithEQPrice.csv')

In [13]:
database[database['Stock']=='MCDOWELL-N']

,Unnamed: 0,Stock,Start Date,End Date,Accord Code,Sr.No.,Company Name,CD_BSE Code,CD_Bse Scrip Name,CD_NSE Symbol,CD_ISIN No


In [14]:
stock_price

,Name,Date,Close,Exchange,Year
13893510,RHI Magnesita India Ltd.,2012-01-01,27.15,NSE,2012
7212596,ICRA Ltd.,2012-01-01,801.90,NSE,2012
9134047,Kemrock Industries & Exports Ltd.,2012-01-01,516.80,NSE,2012
1223778,Apcotex Industries Ltd.,2012-01-01,13.43,NSE,2012
15273135,SRF Ltd.,2012-01-01,47.80,NSE,2012
...,...,...,...,...,...
7731447,Indoco Remedies Ltd.,2022-12-30,406.65,NSE,2022
5954624,Genesys International Corporation Ltd.,2022-12-30,454.70,NSE,2022
8318901,Jainam Ferro Alloys (I) Ltd.,2022-12-30,187.45,NSE,2022
15057697,SAL Steel Ltd.,2022-12-30,15.05,NSE,2022


In [15]:
stock_price[stock_price['Name']=='MCDOWELL_N']

,Name,Date,Close,Exchange,Year
